# Intro to Q\# #
---

## Quantum random number generator

In [1]:
open Microsoft.Quantum.Extensions.Diagnostics;

operation Qrng() : Result {
    using (qubit = Qubit()) {

        H(qubit);
       
        return MResetZ(qubit);
    }
}

Qrng

In [2]:
%simulate Qrng

Zero

In [3]:
open Microsoft.Quantum.Extensions.Diagnostics;
operation QrngWithInfo() : Result {

    using (qubit = Qubit()) {
    
        Message("Here is what the simulator uses to record a qubit in the 0 state:");
        DumpRegister((), [qubit]);

        Message(" ");
        
        H(qubit);
        
        Message("Here is what the simulator uses to record a superposition state:");
        DumpRegister((), [qubit]);
        
        return MResetZ(qubit);
    }
}

QrngWithInfo

In [4]:
%simulate QrngWithInfo

Here is what the simulator uses to record a qubit in the 0 state:
# wave function for qubits with ids (least to most significant): 0
0:	1	0
1:	0	0
 
Here is what the simulator uses to record a superposition state:
# wave function for qubits with ids (least to most significant): 0
0:	0.707106781186548	0
1:	0.707106781186548	0


Zero

In [5]:
operation Qrng(nDigits : Int) : Int[] {

    // Start with an array for the results:
    mutable bitList = new Int[nDigits];
    
    // Loop for nDigits, preparing an entangled state and then measurning it
    for (i in 0..nDigits - 1) {
        using (qubit = Qubit()) {
            H(qubit);
            set bitList[i] = ResultAsInt([MResetZ(qubit)]);
        }  
    }

    return bitList;
}

Qrng

In [6]:
operation Qrng10bit(): Int[] {
    return Qrng(10);
}

Qrng10bit

In [7]:
%simulate Qrng10bit

1, 0, 1, 1, 0, 0, 0, 1, 1, 0

# Deutsch–Jozsa

In [8]:
operation ZeroOracle(control : Qubit, target : Qubit) : Unit {
}

operation OneOracle(control : Qubit, target : Qubit) : Unit {
    X(target);
}

operation IdOracle(control : Qubit, target : Qubit) : Unit {
    CNOT(control, target);
}

operation NotOracle(control : Qubit, target : Qubit) : Unit {
    X(control);
    CNOT(control, target);
    X(control);
}

IdOracle, NotOracle, OneOracle, ZeroOracle

In [9]:
operation IsOracleBalanced( oracle : ((Qubit, Qubit) => Unit)) : Bool {
    mutable result = Zero;
    using ((control, target) = (Qubit(), Qubit())) {
        H(control);
        X(target);
        H(target);
        oracle(control, target);
        H(target); 
        X(target);
        set result = MResetX(control);
    }
    return result == One;
}

IsOracleBalanced

In [10]:
operation IsOZeroOracleBalanced() : Bool{
    return IsOracleBalanced(ZeroOracle);
}

operation IsOneOracleBalanced() : Bool{
    return IsOracleBalanced(OneOracle);
}

operation IsIdOracleBalanced() : Bool{
    return IsOracleBalanced(IdOracle);
}

operation IsNotOracleBalanced() : Bool{
    return IsOracleBalanced(NotOracle);
}

IsIdOracleBalanced, IsNotOracleBalanced, IsOneOracleBalanced, IsOZeroOracleBalanced

In [11]:
%simulate IsOneOracleBalanced

False

In [12]:
%simulate IsOneOracleBalanced

False

In [13]:
%simulate IsIdOracleBalanced

True

In [14]:
%simulate IsNotOracleBalanced

True